In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment. 
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments. 
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

glove_wiki_vectors

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words = ["university", "computer", "virus", "beautiful"]

for w in words:
    print(f"Most similar words to '{w}':")
    print(glove_wiki_vectors.most_similar(w))
    print()

Most similar words to 'university':
[('college', 0.8294211030006409), ('harvard', 0.8156032562255859), ('yale', 0.8113802075386047), ('professor', 0.8103784918785095), ('graduate', 0.7993001341819763), ('faculty', 0.7793422937393188), ('princeton', 0.762718141078949), ('institute', 0.7604438662528992), ('school', 0.7548453211784363), ('stanford', 0.7424825429916382)]

Most similar words to 'computer':
[('computers', 0.8751983046531677), ('software', 0.8373122215270996), ('technology', 0.764215886592865), ('pc', 0.7366448640823364), ('hardware', 0.7290390729904175), ('internet', 0.72867751121521), ('desktop', 0.7234442830085754), ('electronic', 0.7221828699111938), ('systems', 0.7197922468185425), ('computing', 0.7141730189323425)]

Most similar words to 'virus':
[('flu', 0.8397523760795593), ('influenza', 0.8194246292114258), ('h5n1', 0.8128673434257507), ('viruses', 0.7962247133255005), ('infected', 0.7917227745056152), ('infection', 0.7857719659805298), ('disease', 0.7816259264945984

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [7]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [8]:
for w1, w2 in word_pairs:
    sim = glove_wiki_vectors.similarity(w1, w2)
    print(f"Similarity('{w1}', '{w2}') = {sim:.4f}")

Similarity('coast', 'shore') = 0.7000
Similarity('clothes', 'closet') = 0.5463
Similarity('old', 'new') = 0.6432
Similarity('smart', 'intelligent') = 0.7553
Similarity('dog', 'cat') = 0.8798
Similarity('tree', 'lawyer') = 0.0767


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [9]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [10]:
for w in test_words:
    print(w, w in glove_wiki_vectors)

covididiot False
fomo False
frenemies True
anthropause False
photobomb False
selfie False
pxg False
pacg False
cct True
escc True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [13]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [14]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.351824)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [15]:
analogy("man", "engineer", "woman")

man : engineer :: woman : ?


,Analogy word,Score
0,technician,0.662020
1,educator,0.611515
2,engineers,0.592235
3,surgeon,0.588020
4,contractor,0.587330
5,engineering,0.568600
6,nurse,0.556342
7,chemist,0.554719
8,pioneer,0.546008
9,physician,0.545960


In [16]:
print(glove_wiki_vectors.similarity("man", "leader"))
print(glove_wiki_vectors.similarity("woman", "leader"))

0.5314654
0.40278602


In [17]:
print(glove_wiki_vectors.similarity("white", "success"))
print(glove_wiki_vectors.similarity("black", "success"))

0.351824
0.37283087


In [18]:
print(glove_wiki_vectors.similarity("muslim", "terrorist"))
print(glove_wiki_vectors.similarity("christian", "terrorist"))

0.5150928
0.29155728


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

Yes. Even though GloVe Wikipedia embeddings are partially debiased, several harmful patterns remain:

Gender stereotyping:
- Analogies map “woman” to caregiving roles rather than technical roles (e.g., engineer → nurse).

Leadership bias:
- “man” is more similar to “leader” than “woman.”

Race-related bias:
- Words like “white” show higher similarity to positive traits like “success” than “black.”

Religious bias
- Stronger associations between “muslim” and “terrorist” than for other religions.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [21]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [22]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [23]:
def clean_text(text):
    text = text.lower()                           # lowercase
    text = re.sub(r"\s+", " ", text)              # collapse whitespace
    text = re.sub(r"http\S+|www.\S+", " URL ", text)      # replace URLs
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z.-]+\.[a-zA-Z]{2,}", " EMAIL ", text)  # emails
    text = re.sub(r"\d+", " NUM ", text)          # numbers → NUM
    return text.strip()

In [24]:
def preprocess_spacy(text):
    doc = nlp(text)

    tokens = []
    for token in doc:

        if token.is_stop or token.is_punct or token.is_space:
            continue

        if not token.text.isalpha():
            continue

        if token.pos_ not in {"NOUN", "VERB", "ADJ"}:
            continue

        lemma = token.lemma_.strip()
        if len(lemma) < 3:
            continue

        tokens.append(lemma)

    return " ".join(tokens)

In [25]:
import re
from tqdm import tqdm
tqdm.pandas()   # progress bar

df["text_pp"] = (
    df["text"]
      .progress_apply(clean_text)
      .progress_apply(preprocess_spacy)
)

df[["text", "text_pp"]].head()

100%|██████████| 4563/4563 [02:21<00:00, 32.34it/s]


,text,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",know read wonder help provide paragraph unlawf...
1,\n\n\nIt's not a bad question: I don't have an...,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,late spokesman tell journalist close air space...


In [26]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late spokesman tell journalist close air space...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    max_df=0.95,      # ignore extremely common terms
    min_df=5,         # ignore very rare terms
)
X_counts = vectorizer.fit_transform(df["text_pp"])

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

topic_nums = [4, 6, 8, 10]
models = {}

for k in topic_nums:
    lda = LatentDirichletAllocation(
        n_components=k,
        random_state=42,
        learning_method="batch",
    )
    lda.fit(X_counts)
    models[k] = lda
    print(f"k = {k}, log-likelihood = {lda.score(X_counts)}")

k = 4, log-likelihood = -2330596.203649601
k = 6, log-likelihood = -2302951.385000717
k = 8, log-likelihood = -2302173.251189299
k = 10, log-likelihood = -2289315.962782513


To determine an appropriate number of topics k for the LDA model, I compared the log-likelihood values obtained for different values of k. Since log likelihood measures how well the model fits the data, higher log likelihood values (i.e., less negative) indicate a better fit. The results were:
- k = 4: −2,330,596
- k = 6: −2,302,951
- k = 8: −2,302,173
- k = 10: −2,289,315

As k increases, the log-likelihood consistently becomes less negative, meaning the model fit improves with more topics. Based solely on log-likelihood, k = 10 provides the best fit among the tested values.

However, it is important to note that LDA log-likelihood typically increases with larger k, which can lead to overfitting. Therefore, while k = 10 is preferred according to this metric, additional considerations such as topic coherence and interpretability would be necessary to confirm that this value of k produces meaningful and non-redundant topics.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

In [29]:
import numpy as np

def display_topics(model, feature_names, no_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print("  " + " ".join([feature_names[i] 
                               for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [30]:
chosen_k = 10
best_lda = models[chosen_k]

feature_names = vectorizer.get_feature_names_out()
display_topics(best_lda, feature_names)

Topic 0:
  gun file entry firearm weapon use law program section state
Topic 1:
  israeli arab right state jewish pope peace palestinian absolute occupy
Topic 2:
  image file email available program version system use include run
Topic 3:
  book time man people know find day think death say
Topic 4:
  people know think thing good post argument time question way
Topic 5:
  say people come know tell think time see kill happen
Topic 6:
  window widget use application problem display set value file work
Topic 7:
  armenian turkish people government war greek genocide turk turkey village
Topic 8:
  believe think people know christian say religion exist thing atheist
Topic 9:
  game team year play player good win season think get


Topic 1 — Middle East / Israeli–Palestinian Politics
Top words: israeli, arab, right, state, jewish, pope, peace, palestinian, absolute, occupy
Label justification: References to Israeli, Arab, Jewish, and Palestinian indicate geopolitical and cultural discussions about the Middle East conflict.

Topic 2 — Software / Technical Usage Instructions
Top words: image, file, email, available, program, version, system, use, include, run
Label justification: Contains terms about files, programs, versions, and systems, characteristic of technical support or software instructions.

Topic 3 — Literature, Philosophy, and Human Experience
Top words: book, time, man, people, know, find, day, think, death, say
Label justification: Word set reflects reflective or narrative content, often seen in literature or philosophical discussions.

Topic 4 — Online Discussions / Debates
Top words: people, know, think, thing, good, post, argument, time, question, way
Label justification: Very generic debate vocabulary, likely representing general online discussion or opinion threads.

Topic 5 — Crime, Violence, and Events
Top words: say, people, come, know, tell, think, time, see, kill, happen
Label justification: Presence of kill and happen suggests narratives about events or violent incidents, often in news or personal accounts.

Topic 6 — GUI / Software Interface Issues
Top words: window, widget, use, application, problem, display, set, value, file, work
Label justification: Words like window, widget, and display indicate discussions about GUI programming or software interface troubleshooting.

Topic 7 — Historical Conflicts and Genocide
Top words: armenian, turkish, people, government, war, greek, genocide, turk, turkey, village
Label justification: Strong references to Armenian, Turkish, and genocide clearly point to historical conflict discussions.

Topic 8 — Religion and Belief
Top words: believe, think, people, know, christian, say, religion, exist, thing, atheist
Label justification: Terms relate directly to religious belief, atheism, and discussions around the existence of God.

Topic 9 — Sports (Teams and Games)
Top words: game, team, year, play, player, good, win, season, think, get
Label justification: Mentions of team, player, win, season clearly indicate sports-related discussions.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [31]:
doc_topics = best_lda.transform(X_counts)
doc_topics[:5]

array([[0.55169043, 0.00208385, 0.00208384, 0.08527145, 0.19857734,
        0.002084  , 0.0020839 , 0.00208404, 0.15195725, 0.00208389],
       [0.00147076, 0.00147075, 0.00147085, 0.00147102, 0.00147138,
        0.00147091, 0.98676143, 0.00147089, 0.001471  , 0.00147101],
       [0.00200022, 0.00200036, 0.15061173, 0.00200054, 0.83338511,
        0.00200067, 0.00200039, 0.0020003 , 0.00200044, 0.00200023],
       [0.0055561 , 0.00555583, 0.00555634, 0.00555885, 0.00555717,
        0.00555855, 0.94998714, 0.00555687, 0.00555671, 0.00555645],
       [0.00303067, 0.08555601, 0.00303085, 0.00303102, 0.00303066,
        0.00303132, 0.00303078, 0.89019701, 0.00303099, 0.00303067]])

In [32]:
import pandas as pd

df_doc_topics = pd.DataFrame(doc_topics, columns=[f"topic_{i}" for i in range(chosen_k)])
df_doc_topics.head(5)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,0.551690,0.002084,0.002084,0.085271,0.198577,0.002084,0.002084,0.002084,0.151957,0.002084
1,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.986761,0.001471,0.001471,0.001471
2,0.002000,0.002000,0.150612,0.002001,0.833385,0.002001,0.002000,0.002000,0.002000,0.002000
3,0.005556,0.005556,0.005556,0.005559,0.005557,0.005559,0.949987,0.005557,0.005557,0.005556
4,0.003031,0.085556,0.003031,0.003031,0.003031,0.003031,0.003031,0.890197,0.003031,0.003031


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering recommends items based on the similarity between items and the user’s past preferences.
Items are represented using features (e.g., text, genre, keywords), and the system builds a profile of each user by modeling what kinds of features they like. The recommender then suggests new items that are similar to items the user previously interacted with.

2. 	
    1. Filter bubbles / echo chambers: Systems repeatedly show users similar content, reducing exposure to diverse viewpoints.
    2. Amplification of misinformation: Algorithms may promote sensational or extreme content because it increases engagement.

3. Transfer learning in NLP means using knowledge learned from a large pretrained model (such as BERT, GPT, or word embeddings like GloVe) and adapting it to a new, smaller task. Instead of training a model from scratch, we start from a model that has already learned general language structure and only fine-tune it on the target dataset.
This leads to better performance with less data and less computational cost.

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Here is a quick checklist before submitting: 

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](img/eva-well-done.png)